In [2]:
import torch
from torch import nn
from torch.nn  import functional as F
import math

## self attention blockk

In [3]:
class selfAttention(nn.Module):
    def __init__(self,n_heads:int,d_embed:int, in_proj_bias=True ,out_proj_bias=True):
        super.__init__()
        self.in_proj=nn.Linear(d_embed,3*d_embed,bias=in_proj_bias)
        self.out_proj=nn.Linear(d_embed,d_embed,bias=out_proj_bias)
        self.heads=n_heads
        self.d_head=d_embed//n_heads

    def forward(self, x, casual_mask=False):
        # x: (Batch_size,seq_length (len_of_embed),Dim)
        input_shape=x.shape
        batch_size,seq_len,d_embed=input_shape

        # required for converting the input into desired input for K,q,v
        intermin_shape=(batch_size,seq_len,self.heads,self.d_head)

        # (Batch_size,seq_length (len_of_embed),Dim)--> (Batch_size,seq_length (len_of_embed),Dim*3 )--> 3 tensors of shape  (Batch_size,seq_length (len_of_embed),Dim)
        k,q,v=self.in_proj(x).chunk(3,dim=-1) # to the last dim which is itself in this case is "DIM"

        # splitting the number of k,q,v in n_heads


# transpose(): Swaps two dimensions of a tensor, effectively changing the arrangement of the data.
# view(): Reshapes the tensor without changing its data. The number of elements must remain the same.

#Creating Q, K, V:
# For each word, we create Q, K, and V (in practice, these are learned linear transformations of the embeddings).

        q=q.view(intermin_shape).transpose(1,2)  #(Batch_size,seq_length,Dim )--> (Batch_size,seq_length, Heads, DIM/Heads )
        k=k.view(intermin_shape).transpose(1,2)  #(Batch_size,seq_length,Dim )--> (Batch_size,seq_length, Heads, DIM/Heads )
        v=v.view(intermin_shape).transpose(1,2) #(Batch_size,seq_length,Dim )--> (Batch_size,seq_length, Heads, DIM/Heads )


        # doing the weights 
        weights= q@k.transpose(-1,-2)

        if casual_mask: # so that we can disaacoatiate simliars tokens nex to each other, makes a -infity passed to softmax--> gives 0
            # mask where the upper traingles are 1 above the principal Diagonal
            mask=torch.ones_like(weights,dtype=torch.bool).triu(1)
            weights.masked_fill(mask,-torch.inf)

        weights/=math.sqrt(self.d_head)
        weights=F.softmax(weights,dim=-1)

        output=weights@v # (Batch_size,Heads,Seq_len,Seq_len ) @ (Batch_size,Heads,Seq_len,DIM/Heads ) --> (Batch_size,Heads,Seq_len,DIM/Heads )
        
        output.transpose(1,2) #m (Batch_size , Seq_len, Heads, DIM/Heads )

        output=output.reshape(input_shape)

        output=self.out_proj(output)

        # (Batch_size,seq_length,Dim )
        return output


## Adding Clip Encoder code
